# Preprocessing

Preprocessing will revolve around the following steps: 
1. Draft Dataset 
- Imputation of missing variables 
- Standardisation of attirbutes
- Setting variable types
- setting positions

This current notebook works, but has a few bugs/cases to fix such as:
- Playing no games for the first club, and some for the second 
- Players who played in 2010-2012, but not in 2013 (the cut off year)
- Changed clubs in 2013 and played no more games 
- Probably more

In [1]:
import pandas as pd
import re
# Load the datasets
draft_data = pd.read_csv('../data/raw/draft_history.csv',  index_col=[0])
position_history = pd.read_csv('../data/raw/game_positions.csv',  index_col=[0])
player_stats = pd.read_csv('../data/raw/player_statistics.csv',  index_col=[0])
afltable_hist = pd.read_csv('../data/raw/afltables_draft.csv',  index_col=[0])

# Constant Variables

# This refers to the year of which player stats were first collected
STATISTICS_COLLECTION_YEAR = '2010'
# This refers to the year of which draft history was first captured 
DRAFT_START_YEAR = '2013'

/tmp/ipykernel_317/714907168.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  position_history = pd.read_csv('../data/raw/game_positions.csv',  index_col=[0])


In [2]:
# Before anything, we need to join the afltable_hist and draft_data to get all the attributes we desire

# This function standardises team names to be used as a attribute to join the 2 different draft datasets
def team_standardise(team):
    team_dict = {'Adelaide' : 'Adelaide Crows',
                  'GWS'  :  'GWS Giants',
                  'Gold Coast' : 'Gold Coast Suns',
                  'Geelong' : 'Geelong Cats',
                  'Sydney' : 'Sydney Swans',
                  'West Coast' : 'West Coast Eagles'}
    if team in team_dict.keys():
        team = team_dict[team]
    return team
    
# This function determines the last playing year of a player
def last_season(seasons):
    seasons = re.sub(r'[ |\-|,|A-z]', '', seasons)
    # take last 4 didgets
    return seasons[-4:]

# This function determines the first playing year of a player
def first_season(seasons):
    seasons = re.sub(r'[ |\-|,|A-z]', '', seasons)
    # take last 4 didgets
    return seasons[:4]

# This finds the last year a player plays over their whole career
def players_last_year(player, HT, WT, dataset = afltable_hist):
    # frst take subset with player information
    player_draft_sub = dataset[(dataset['Player'] == player) & (dataset['HT'] == HT) & (dataset['WT'] == WT)] 
    # if there is only one row, just output the club
    most_recent_year = '0'
    for history in player_draft_sub['Seasons']:
        # work out if 2013 lies before the last year
        years = last_season(history)
        if most_recent_year < years:
            most_recent_year = years
    return most_recent_year

# This finds the club the player played for in 2013 (for afltables_hist)
def players_match_year(player, HT, WT, dataset = afltable_hist, year = DRAFT_START_YEAR):
    # frst take subset with player information
    player_draft_sub = dataset[(dataset['Player'] == player) & (dataset['HT'] == HT) & (dataset['WT'] == WT)] 
    # sort the subset in order of clubs played for
    player_draft_sub.sort_values('First_season_club', inplace=True)
    no_clubs = len(player_draft_sub)
    player_draft_sub = player_draft_sub.reset_index()
    # if there is only one row, just output the club
    for club in range(0, no_clubs):
        subset = player_draft_sub.iloc[club]
        # work out if 2013 lies before the last year
        if subset['Last_season_club'] >= year:
            output = subset['Team']
            return output
    return 'Unknown'

# This is the same function as above, but for the (draft_data) dataset
def key_year_extract(id, dataset = draft_data):
    # frst take subset with player information
    player_draft_sub = dataset.loc[dataset['id'] == id] 
    # sort the subset in order of clubs played for
    player_draft_sub.sort_values('season', inplace=True)
    # now just grab the first entry
    player_draft_sub = player_draft_sub.reset_index()
    return player_draft_sub['team'].values[0]

# ensure the names are standardised to ensure compadability between datasets
def name_cleanse(name):
    # need to remove the middle names from players
    name = re.sub(' [A-z]\. ', ' ', name)
    # need to keep the first 2 letters of the first name
    first_n, last_n = name.split(' ', 1)
    # replace all (tom)s with thomas
    first_n = re.sub('[\W_| ]+', '', first_n.lower())
    if first_n in ('tom', 'tommy'):
        first_n = 'thomas'
    last_n = re.sub('[\W_| ]+', '', last_n.lower())
    return first_n[0:2] + ' ' + last_n

In [3]:
# First remove players whom played only in seasons before and including 2010 (want at least a year to compare to: 2010 and 2011)
afltable_hist['Last_season_club'] = afltable_hist['Seasons'].apply(lambda x: last_season(x))
afltable_hist['First_season_club'] = afltable_hist['Seasons'].apply(lambda x: first_season(x))
# remove players who finished at a club before 1995
afltable_hist = afltable_hist[afltable_hist['First_season_club'] > '1990']
# The reason we need to use the HT and WT as input, is to differenciate between players of the same name
afltable_hist['Last_season'] = afltable_hist.apply(lambda x: players_last_year(x['Player'], x['HT'], x['WT']), axis=1)
# Want only players who are registered to an afl club from 2013 onwards
afltable_hist = afltable_hist[afltable_hist['Last_season'] >= DRAFT_START_YEAR]
afltable_hist['Team'] = afltable_hist['Team'].apply(lambda x: team_standardise(x))
afltable_hist['Player_alt'] = afltable_hist['Player'].apply(lambda x: name_cleanse(x))
# for the join club
afltable_hist = afltable_hist.reset_index()
afltable_hist['Key_club'] = afltable_hist.apply(lambda x: players_match_year(x['Player'], x['HT'], x['WT'], afltable_hist), axis=1)
# sort again
afltable_hist.sort_values('First_season_club', inplace=True)

/tmp/ipykernel_317/1766433261.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afltable_hist['Last_season'] = afltable_hist.apply(lambda x: players_last_year(x['Player'], x['HT'], x['WT']), axis=1)


In [4]:
# Now lets obtain the desired aggreagated statistics
afltable_agg = afltable_hist.groupby(['Player_alt', 'Key_club']).agg(Games = pd.NamedAgg('Games', 'sum'), 
                                                  Clubs = pd.NamedAgg('Team', aggfunc=lambda x: list(x)), 
                                                  Total_Clubs = pd.NamedAgg('Team', aggfunc=lambda x: len(list(x))),
                                                  Goals = pd.NamedAgg('Goals', 'sum'), 
                                                  Debut_season = pd.NamedAgg('First_season_club', 'min'))

In [5]:
afltable_agg

Games  \
Player_alt   Key_club                   
aa black     North Melbourne       57   
             West Coast Eagles      1   
aa cornelius Brisbane Lions        25   
aa davey     Melbourne            178   
aa edwards   Richmond              94   
...                               ...   
za smith     Gold Coast Suns      124   
za sproule   GWS Giants            17   
za trew      West Coast Eagles      2   
za tuohy     Carlton              250   
za williams  GWS Giants           136   

                                                                         Clubs  \
Player_alt   Key_club                                                            
aa black     North Melbourne                   [North Melbourne, Geelong Cats]   
             West Coast Eagles                             [West Coast Eagles]   
aa cornelius Brisbane Lions                                   [Brisbane Lions]   
aa davey     Melbourne                                             [Melbourne]   
aa edwards   Richmond           [West Coast Eagles, North Melbourne, Richmond]   
...                                                                        ...   
za smith     Gold Coast Suns                   [Gold Coast Suns, Geelong Cats]   
za sproule   GWS Giants                                           [GWS Giants]   
za trew      West Coast Eagles                             [West Coast Eagles]   
za tuohy     Carlton                                   [Carlton, Geelong Cats]   
za williams  GWS Giants                                  [GWS Giants, Carlton]   

                                Total_Clubs  Goals Debut_season  
Player_alt   Key_club                                            
aa black     North Melbourne              2   69.0         2011  
             West Coast Eagles            1    1.0         2022  
aa cornelius Brisbane Lions               1   35.0         2009  
aa davey     Melbourne                    1  174.0         2004  
aa edwards   Richmond                     3  139.0         2005  
...                                     ...    ...          ...  
za smith     Gold Coast Suns              2   54.0         2011  
za sproule   GWS Giants                   1   13.0         2019  
za trew      West Coast Eagles            1    1.0         2022  
za tuohy     Carlton                      2   87.0         2011  
za williams  GWS Giants                   2   32.0         2013  

[1491 rows x 5 columns]

In [6]:
afltable_hist[afltable_hist['Key_club'] == 'Brisbane Lions']

,index,Player,Team,Cap,#,HT,WT,Games,Wins,Draws,...,Goals,Seasons,Debut,Last,date_accessed,Last_season_club,First_season_club,Last_season,Player_alt,Key_club
106,292,Simon Black,Brisbane Lions,39,20.0,186cm,81kg,322,174,4,...,171.0,1998-2013,18y 359d,34y 136d,2022-09-27,2013,1998,2013,si black,Brisbane Lions
107,309,Jonathan Brown,Brisbane Lions,56,16.0,195cm,103kg,256,145,3,...,594.0,2000-2014,18y 162d,32y 228d,2022-09-27,2014,2000,2014,jo brown,Brisbane Lions
108,319,Ashley McGrath,Brisbane Lions,66,9.0,183cm,86kg,214,103,2,...,170.0,2001-2014,18y 91d,31y 96d,2022-09-27,2014,2001,2014,as mcgrath,Brisbane Lions
1592,13217,Matt Maguire,St Kilda,1489,31.0,192cm,101kg,99,54,1,...,19.0,2002-2008,17y 338d,23y 339d,2022-09-27,2008,2002,2015,ma maguire,Brisbane Lions
1811,14954,Brent Staker,West Coast Eagles,150,41.0,196cm,98kg,110,64,1,...,84.0,2003-2009,19y 63d,25y 97d,2022-09-27,2009,2003,2015,br staker,Brisbane Lions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,476,Kai Lohmann,Brisbane Lions,223,1.0,185cm,77kg,2,2,0,...,NaN,2022,18y 343d,18y 353d,2022-09-27,2022,2022,2022,ka lohmann,Brisbane Lions
223,477,James Tunstill,Brisbane Lions,224,29.0,187cm,75kg,3,1,0,...,1.0,2022,18y 340d,18y 357d,2022-09-27,2022,2022,2022,ja tunstill,Brisbane Lions
224,478,Carter Michael,Brisbane Lions,225,39.0,188cm,83kg,1,0,0,...,1.0,2022,20y 49d,20y 49d,2022-09-27,2022,2022,2022,ca michael,Brisbane Lions
225,479,Darcy Wilmot,Brisbane Lions,226,44.0,183cm,75kg,3,2,0,...,1.0,2022,18y 244d,18y 259d,2022-09-27,2022,2022,2022,da wilmot,Brisbane Lions


In [7]:
# Before anything, we need to join the afltable_hist and draft_data to get all the attributes we desire
# get common name attribute
draft_data['Player'] = draft_data['firstName'] + ' ' + draft_data['surname']
draft_data['Player_alt'] = draft_data['Player'].apply(lambda x: name_cleanse(x))
# need to setup Key_club (which is the first club a player is played at since 2013)
draft_data['Key_club'] = draft_data['id'].apply(lambda x: key_year_extract(x))

/tmp/ipykernel_317/3177397332.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_draft_sub.sort_values('season', inplace=True)
/tmp/ipykernel_317/3177397332.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_draft_sub.sort_values('season', inplace=True)
/tmp/ipykernel_317/3177397332.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_draft_sub.sort_values('season', inplace=True)
/tmp/ipykernel_317/3177397332.p

In [8]:
draft_data

,firstName,surname,id,team,season,jumperNumber,position,providerId,dateOfBirth,draftYear,heightInCm,weightInKg,recruitedFrom,debutYear,draftType,draftPosition,data_accessed,Player,Player_alt,Key_club
1,Scott,Thompson,2,Adelaide Crows,2013,5,MIDFIELDER,CD_I210016,1983-03-14,2000.0,185,86,North Haven (SA)/Port Adelaide (SANFL)/Melbourne,2001.0,nationalDraft,16.0,2022-08-26,Scott Thompson,sc thompson,Adelaide Crows
2,Graham,Johncock,1132,Adelaide Crows,2013,18,MEDIUM_FORWARD,CD_I210056,1982-10-21,2000.0,178,82,Port Adelaide (SANFL),2002.0,nationalDraft,67.0,2022-08-26,Graham Johncock,gr johncock,Adelaide Crows
3,Brent,Reilly,31,Adelaide Crows,2013,3,MEDIUM_DEFENDER,CD_I220086,1983-11-12,2001.0,183,84,Gisborne (Vic)/Calder U18,2002.0,nationalDraft,12.0,2022-08-26,Brent Reilly,br reilly,Adelaide Crows
4,Ben,Rutten,43,Adelaide Crows,2013,25,KEY_DEFENDER,CD_I220093,1983-05-28,2002.0,190,98,West Adelaide (SA),2003.0,rookieElevation,NaN,2022-08-26,Ben Rutten,be rutten,Adelaide Crows
5,Jason,Porplyzia,3,Adelaide Crows,2013,40,MEDIUM_FORWARD,CD_I230174,1984-11-27,2005.0,179,80,West Adelaide (SA),2006.0,preseason,9.0,2022-08-26,Jason Porplyzia,ja porplyzia,Adelaide Crows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8183,Josh,Rotham,1380,West Coast Eagles,2022,35,MEDIUM_DEFENDER,CD_I997823,1998-02-25,2016.0,193,89,Whitford (WA)/West Perth (WAFL),2019.0,nationalDraft,37.0,2022-08-26,Josh Rotham,jo rotham,West Coast Eagles
8184,Alex,Witherden,1307,West Coast Eagles,2022,23,MEDIUM_DEFENDER,CD_I998128,1998-09-10,2016.0,188,86,St Joseph&apos;s (Vic)/Geelong U18/Brisbane,2017.0,nationalDraft,23.0,2022-08-26,Alex Witherden,al witherden,Brisbane Lions
8185,Luke,Strnadica,1391,West Coast Eagles,2022,45,RUCK,CD_I998192,1998-01-01,2021.0,202,102,Winnacott (WA)/ E Frem (WAFL)/Fremantle list/E...,2022.0,preseason,NaN,2022-08-26,Luke Strnadica,lu strnadica,Fremantle
8186,Jamaine,Jones,1300,West Coast Eagles,2022,31,MEDIUM_DEFENDER,CD_I998791,1998-09-29,2019.0,179,81,Heywood (Vic)/Portland (Vic)/North Ballarat U1...,2018.0,preseason,NaN,2022-08-26,Jamaine Jones,ja jones,Geelong Cats


In [9]:
draft_data_agg = draft_data.groupby(['Player_alt', 'Key_club']).agg(id = pd.NamedAgg('id', lambda x: pd.Series.mode(x)[0]),
                                                                    player_name = pd.NamedAgg('Player', lambda x: pd.Series.mode(x)),
                                                                    first_reg_season = pd.NamedAgg('season', 'min'),
                                                                    Last_reg_season = pd.NamedAgg('season', 'max'),
                                                                    Position = pd.NamedAgg('position', lambda x: set(x)),
                                                                    providerId = pd.NamedAgg('providerId', lambda x: pd.Series.mode(x)[0]),
                                                                    dateOfBirth = pd.NamedAgg('dateOfBirth', lambda x: pd.Series.mode(x)[0]),
                                                                    draftYear = pd.NamedAgg('draftYear', 'mean'),
                                                                    heightInCm = pd.NamedAgg('heightInCm', lambda x: pd.Series.mode(x)[0]),
                                                                    weightInKg = pd.NamedAgg('weightInKg', lambda x: pd.Series.mode(x)[0]),
                                                                    recruitedFrom = pd.NamedAgg('recruitedFrom', lambda x: pd.Series.mode(x)),
                                                                    debutYear = pd.NamedAgg('debutYear', 'mean'),
                                                                    draftType = pd.NamedAgg('draftType', lambda x: pd.Series.mode(x)),
                                                                    draftPosition = pd.NamedAgg('draftPosition', 'mean'),
                                                                    
                                                                    )

In [10]:
full_draft = draft_data_agg.join(afltable_agg)

In [11]:
full_draft

id      player_name  first_reg_season  \
Player_alt   Key_club                                                     
aa black     North Melbourne     532      Aaron Black              2013   
             West Coast Eagles  6203      Aaron Black              2022   
aa cornelius Brisbane Lions     1071  Aaron Cornelius              2013   
aa davey     Melbourne          1075      Aaron Davey              2013   
aa edwards   Richmond            251    Aaron Edwards              2013   
...                              ...              ...               ...   
za trew      West Coast Eagles  4076        Zane Trew              2021   
za tuohy     Carlton              73       Zach Tuohy              2013   
za webster   Hawthorn            257      Zac Webster              2014   
za williams  GWS Giants          483     Zac Williams              2013   
             Geelong Cats       8397    Zane Williams              2022   

                                Last_reg_season  \
Player_alt   Key_club                             
aa black     North Melbourne               2018   
             West Coast Eagles             2022   
aa cornelius Brisbane Lions                2013   
aa davey     Melbourne                     2013   
aa edwards   Richmond                      2014   
...                                         ...   
za trew      West Coast Eagles             2022   
za tuohy     Carlton                       2022   
za webster   Hawthorn                      2016   
za williams  GWS Giants                    2022   
             Geelong Cats                  2022   

                                                              Position  \
Player_alt   Key_club                                                    
aa black     North Melbourne                    {KEY_FORWARD, UNKNOWN}   
             West Coast Eagles                        {MEDIUM_FORWARD}   
aa cornelius Brisbane Lions                              {KEY_FORWARD}   
aa davey     Melbourne                                {MEDIUM_FORWARD}   
aa edwards   Richmond                        {UNKNOWN, MEDIUM_FORWARD}   
...                                                                ...   
za trew      West Coast Eagles        {MIDFIELDER, MIDFIELDER_FORWARD}   
za tuohy     Carlton            {MIDFIELDER, MEDIUM_DEFENDER, UNKNOWN}   
za webster   Hawthorn                                        {UNKNOWN}   
za williams  GWS Giants                     {MEDIUM_DEFENDER, UNKNOWN}   
             Geelong Cats                             {MEDIUM_FORWARD}   

                                 providerId dateOfBirth  draftYear  \
Player_alt   Key_club                                                
aa black     North Melbourne     CD_I291672  1990-11-29     2009.0   
             West Coast Eagles   CD_I291589  1992-12-25     2021.0   
aa cornelius Brisbane Lions      CD_I261391  1990-05-29     2008.0   
aa davey     Melbourne           CD_I200002  1983-06-10     2003.0   
aa edwards   Richmond            CD_I230240  1984-03-02     2006.0   
...                                     ...         ...        ...   
za trew      West Coast Eagles  CD_I1011449  2002-04-26     2020.0   
za tuohy     Carlton             CD_I292511  1989-12-10     2012.0   
za webster   Hawthorn            CD_I296232  1995-08-11     2014.0   
za williams  GWS Giants          CD_I294685  1994-09-20     2013.0   
             Geelong Cats       CD_I1013038  2001-06-05     2022.0   

                                heightInCm  weightInKg  \
Player_alt   Key_club                                    
aa black     North Melbourne           192          85   
             West Coast Eagles         181          81   
aa cornelius Brisbane Lions            194          91   
aa davey     Melbourne                 177          76   
aa edwards   Richmond                  184          93   
...                                    ...         ...   
za trew      West Coast Eagles         187          81  

In [12]:
full_draft[(full_draft['debutYear'].notna()) & (full_draft['Total_Clubs'].isna())].size

912

In [13]:
afltable_hist[afltable_hist['Player'] == 'Alec Waterman']

,index,Player,Team,Cap,#,HT,WT,Games,Wins,Draws,...,Goals,Seasons,Debut,Last,date_accessed,Last_season_club,First_season_club,Last_season,Player_alt,Key_club
579,4208,Alec Waterman,Essendon,1177,45.0,183cm,89kg,22,9,0,...,27.0,2021-2022,24y 227d,25y 295d,2022-09-27,2022,2021,2022,al waterman,Essendon


In [15]:
full_draft[full_draft['player_name'] == 'Shaun McKernan']

,,id,player_name,first_reg_season,Last_reg_season,Position,providerId,dateOfBirth,draftYear,heightInCm,weightInKg,recruitedFrom,debutYear,draftType,draftPosition,Games,Clubs,Total_Clubs,Goals,Debut_season
Player_alt,Key_club,,,,,,,,,,,,,,,,,,,
sh mckernan,Adelaide Crows,17,Shaun McKernan,2013,2021,"{KEY_FORWARD, UNKNOWN, RUCK}",CD_I280038,1990-09-01,NaN,198,98,[],NaN,[],NaN,91.0,"[Adelaide Crows, Essendon, St Kilda]",3.0,74.0,2009


# 1. Draft History